In [1]:
import os 
os.chdir('../')
%pwd

'd:\\Projects\\Deep_learning\\End_to_End_Deep_learning_Kidney_tumour_classification_MLFOW_DVC'

In [2]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/senthil100695/End_to_End_Deep_learning_Kidney_tumour_classification_MLFOW_DVC.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "senthil100695"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "c6a12eba9766266ebce1f50a4d02ef18796984a2"

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model('artifacts/training/model.h5')


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class EvaluationConfig:
    path_to_model : Path
    training_data : Path
    all_params : dict
    mlflow_uri : str
    params_image_size : list
    params_batch_size : int
    

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) ->EvaluationConfig:
        eval_config = EvaluationConfig(
            path_to_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/kidney-ct-scan-image",
            all_params = self.params,
            mlflow_uri = "https://dagshub.com/senthil100695/End_to_End_Deep_learning_Kidney_tumour_classification_MLFOW_DVC.mlflow",
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )

        return eval_config

In [11]:
import os
import urllib.request as request
import zipfile
import mlflow 
import mlflow.keras
from urllib.parse import urlparse

In [17]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset= 'validation',
            shuffle= False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {'loss': self.score[0], "accuracy": self.score[1]}
        save_json(path=Path('scores.json'), data = scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {'loss' : self.score[0], "accuracy" : self.score[1]}
            )

            #model registry does not  work with file store

            if tracking_url_type_store != 'file':
                mlflow.keras.log_model(self.model, "model", registered_model_name='VGG16Model')

            else:
                mlflow.keras.log_model(self.model, 'model')


In [18]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(config=evaluation_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2023-10-10 12:40:49,829: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-10 12:40:55,064: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-10 12:40:56,650: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 465s 46s/step - loss: 15.5071 - accuracy: 0.5180
[2023-10-10 12:50:30,111: INFO: common: json file saved at: scores.json]
[2023-10-10 12:51:34,868: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1129)'))': /senthil100695/End_to_End_Deep_learning_Kidney_tumour_classification_MLFOW_DVC.mlflow/api/2.0/mlflow/runs/create]


2023/10/10 12:51:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-10-10 12:52:29,544: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\senthil\AppData\Local\Temp\tmpc_gkvhf7\model\data\model\assets
[2023-10-10 12:52:50,829: INFO: builder_impl: Assets written to: C:\Users\senthil\AppData\Local\Temp\tmpc_gkvhf7\model\data\model\assets]


2023/10/10 12:55:20 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\senthil\AppData\Local\Temp\tmpc_gkvhf7\model, flavor: tensorflow), fall back to return ['tensorflow==2.12.0']. Set logging level to DEBUG to see the full traceback.
c:\Users\senthil\anaconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[2023-10-10 12:57:44,840: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /senthil100695/End_to_End_Deep_learning_Kidney_tumour_classification_MLFOW_DVC.mlflow/api/2.0/mlflow/runs/get?run_uuid=d50085459e72489090c6139bff7c8e39&run_id=d50085459e72489090c6139bff7c8e39]
[2023-10-10 13:02:24,692: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /senthil100695/End_to_End_Deep_learning_Kidney_tumour_classification_MLFOW_DVC.mlflow/api/2.0/mlflow-artifacts/artifacts/4071990bd070440c9aa3990e3c52497c/d50085459e72489090c6139bff7c8e39/artifacts/model/data/model/variables/variables.data-00000-of-00001]
[2023-10-10 13:02:30,028: WARNING: con


KeyboardInterrupt

